In [87]:
%load_ext autoreload
%autoreload 2
from utilities.circuit_basics import Evaluator
evaluator = Evaluator(loading=True, args={"n_qubits":4, "J":0.0})
unitary, energy, indices, resolver = evaluator.raw_history[len(list(evaluator.raw_history.keys()))-1]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
unitary

(0, 0): ───Rx(-π)─────────────────────@───@───────────────────────────@────────────Rx(π)──────────────────────────────────────
                                      │   │                           │
(0, 1): ───Rx(0)──────────────────────┼───┼───────────────────────────┼───────────────────────────────────────────────────────
                                      │   │                           │
(0, 2): ──────────────────────────────┼───X─────────────Rz(0.161π)────X────────────@──────────────────────────────────@───X───
                                      │                                            │                                  │   │
(0, 3): ───Rx(-0.69π)───Rz(-0.496π)───X───Rz(-0.639π)───Rx(-0.505π)───Rz(0.398π)───X───────Rz(0.424π)───Rx(-0.237π)───X───@───

In [89]:
from utilities.variational import VQE
from utilities.circuit_basics import Evaluator
from utilities.idinserter import IdInserter
from utilities.simplifier import Simplifier
from utilities.unitary_killer import UnitaryMurder

In [90]:
iid = IdInserter(n_qubits=evaluator.n_qubits)
#Simplifier reduces gates number as much as possible while keeping same expected value of target hamiltonian
Simp = Simplifier(n_qubits=evaluator.n_qubits)


In [91]:
indices

[16, 19, 15, 2, 15, 19, 15, 1, 14, 1, 16, 17, 8, 15, 19, 8, 11]

In [92]:
qubit=3
block_to_insert = [evaluator.number_of_cnots+(p*evaluator.n_qubits)+qubit for p in [0,1,0]]
M_indices, M_symbols_to_values, M_idx_to_symbols = iid.place_almost_identity(indices, resolver, random=False, block_to_insert=block_to_insert, insertion_index=3)
evaluator.give_unitary(M_indices,M_symbols_to_values)

(0, 0): ───Rx(-π)─────────────────────────────────────────────────────────────@───@───────────────────────────@────────────Rx(π)──────────────────────────────────────
                                                                              │   │                           │
(0, 1): ───Rx(0)──────────────────────────────────────────────────────────────┼───┼───────────────────────────┼───────────────────────────────────────────────────────
                                                                              │   │                           │
(0, 2): ──────────────────────────────────────────────────────────────────────┼───X─────────────Rz(0.161π)────X────────────@──────────────────────────────────@───X───
                                                                              │                                            │                                  │   │
(0, 3): ───Rx(-0.69π)───Rz(-0.496π)───Rz(0.032π)───Rx(0.032π)───Rz(-0.032π)───X───Rz(-0.639π)───Rx(-0.505π)───Rz(0.398π)───X───────Rz(0.424π)───Rx(-0.237π)───X───@───

In [93]:
from tqdm import tqdm
for k in tqdm(range(10**2)):
    Sindices, Ssymbols_to_values, Sindex_to_symbols = Simp.reduce_circuit(M_indices, M_symbols_to_values, M_idx_to_symbols)

100%|██████████| 100/100 [00:10<00:00,  9.21it/s]


In [54]:
from functools import wraps
import errno
import os
import signal
from datetime import datetime

class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result
        return wraps(func)(wrapper)
    return decorator

In [55]:
@timeout(1)
def che():
    st = datetime.now()
    while True and (datetime.now()-st).total_seconds()<3:
        pass

In [56]:
che()

TimeoutError: Timer expired